## Step-by-step guide to generating an interactive climate map in Bokeh (& Geopandas)

- **With some specific boilerplate code already filled in.**
    

- **CREDITS**:

    - The idea / code for this lesson was heavily inspired by the following tutorial:
        - [A Complete Guide to an Interactive Geographical Map using Python](https://towardsdatascience.com/a-complete-guide-to-an-interactive-geographical-map-using-python-f4c5197e23e0), by [Shivangi Patel](https://github.com/CrazyDaffodils).
    
    - The tutorial was adapted / re-written by [Paul Wlodkowski](https://github.com/pawlodkowski) for the *Plotting on Maps* lesson @ Spiced Academy.
        - The data for this particular lesson was scraped from [Berkeley Earth](http://berkeleyearth.lbl.gov/country-list/) and cleaned / pre-processed ahead of time.

<img src="../img/bokeh.jpg" align="center" alt="bokeh_map" width="600/"/>

- **Make sure you already have bokeh and geopandas installed**! (e.g. `pip` or `conda`), e.g.:

    - `pip install bokeh==2.1.1`
    
    - `pip install geopandas==0.8.0`
    

*Note*: this notebook was last run and tested on bokeh version 2.2.0 (released June 22, 2020) and geopandas version 0.8.0 (released July 15, 2020).

---

### STEP 1: Read in historical temperature data
- Historical temperature data scraped for all countries from [Berkeley Earth](http://berkeleyearth.lbl.gov/country-list/) 
- According to Berkeley Earth: "Temperatures are in Celsius and reported as anomalies relative to the Jan 1951-Dec 1980 average."

In [ ]:
DATA = '../data/all_country_temp_data_CLEAN.csv'

In [ ]:
#Read in the data with pandas
import pandas as pd

df = pd.read_csv(DATA)

---
---

### STEP 2: Read in the geographic data (geometric shapes of all countries in the world) 
- Hint: Use GeoPandas
    - What is a **Shape file (.shp)?**
        - https://en.wikipedia.org/wiki/Shapefile#Shapefile_shape_format_(.shp)
- Publicly available GIS data downloaded from *Natural Earth*: https://www.naturalearthdata.com/downloads/110m-cultural-vectors/

In [ ]:
SHAPEFILE = '../data/ne_110m_admin_0_countries.shp'

In [ ]:
#Read in the shapefile with geopandas
import geopandas as gpd
gdf = gpd.read_file(___)

---
---

### STEP 3: Group / aggregate the temperature anomaly data by country, year
- For simplicity, we're only interested in yearly averages

### STEP 4: Merge Data Sets.
- We want to have our temperature data and geometric data in one place.
- Make sure you're still left with a GeoDataFrame at the end.

---
---
### Time for Visualization with

<img src="https://static.bokeh.org/logos/logotype.svg" align="left" alt="bokeh_logo" width="200/"/>

---
---

### STEP 5: Plot data on a map for a single year.
- We'll make it interactive later
- The Bokeh library (as well as many other JavaScript-based mapping libraries) requires the data to be in GeoJSON format.
    - **HINT**: How can you convert a GeoDataFrame into a GeoJSON? Think of how you might do this in regular pandas.



   ### 5a. Generate a blank canvas / figure.

In [ ]:
from bokeh.plotting import figure

In [ ]:
p = figure(title = _____,
           plot_height = 600,
           plot_width = 1000,
          )

In [ ]:
#Display figure inline in Jupyter Notebook.
from bokeh.io import output_notebook, show

In [ ]:
output_notebook()
show(p)

---

### 5b. Generate a GeoJSON for a single year and use it to add shapes onto the figure
- Let's use the year 2000 as an example.
- **Programming Tip**: 
    - If we can write code to work for a single year (hardcoded), then we can generalize this later to work for *any*  year!

In [ ]:
from bokeh.models import GeoJSONDataSource

In [ ]:
gdf_2000 = gdf[gdf['____'] == ____]
json_2000 = gdf_2000.to_____()

In [ ]:
geosource = GeoJSONDataSource(geojson = _____)

In [ ]:
#Add patch renderer to figure. i.e. actually add the map to the canvas.
p.patches('xs',
          'ys',
          source = _____,
          line_color = '_______',
          line_width = 0.25)

---

### 5c. Associate temperature values with colors
- https://docs.bokeh.org/en/latest/docs/reference/palettes.html#brewer-palettes

In [ ]:
from bokeh.palettes import brewer

In [ ]:
palette = brewer['____']_____
#Hint: what data type is "brewer"? How can we parse it?

In [ ]:
from bokeh.models import LinearColorMapper

In [ ]:
color_mapper = LinearColorMapper(palette = _____,
                                 low = _____,
                                 high = _____, 
                                 nan_color = ______)

In [ ]:
from bokeh.models import ColorBar
color_bar = ColorBar(color_mapper = _______,
                     label_standoff = 8,
                     width = ______,
                     height = ______,
                     location = (0,0),
                     orientation = 'horizontal'
                    )

In [ ]:
p.add_layout(_______, 'below')

In [ ]:
p.patches('xs',
          'ys',
          source = _______,
          fill_color = {'field' :'_______', 'transform': color_mapper}, ### NEW ###
          line_color = '______',
          line_width = _______)

*Note*: The fill-in-the-blank for `fill_color` is not obvious. Here's a hint:
- For "field", if you examine the GeoJson object you created earlier, what is the name of the key within the "properties" dictionary that corresponds to the numerical value for which you would like to map to the color mapper?
- Which attribute of our `geosource` variable allows us to view / access the original geojson information?
    - Use `dir()` or the `TAB` key in Jupyter notebook to see the attributes of any python variable!

In [ ]:
show(p)

---
---

### STEP 6: Add interactivity so that we can change attributes of the map with a slider
- Bokeh provides an extensive set of widgets and tools and makes it very simple to create rich, interactive visualizations.
- Define a couple functions and combine it with the code you've already written for creating the static map.

---

### 6a. Make our lives easier by defining a function that changes the source data based on year.

In [ ]:
def get_geojson(___):
    """Input a year (int) and return corresponding GeoJSON"""
    gdf_year = gdf[gdf['____'] == ___] 
    return gdf_year.______

geosource = GeoJSONDataSource(geojson = get_geojson(_____))

---

### 6b. Add a slider widget
- We'd like to add a tool that we can interactively slide to change the year displayed on the map.

In [ ]:
from bokeh.models import Slider

In [ ]:
slider = Slider(title = '_____', start = _____, end = _____, step = _____, value = _____)
#define the constraints of the year slider

---

### 6c. Write a "callback" function that defines what happens whenever we move the slider.
- This part is tricky; you're going to have to use your knowledge of python objects and namespaces (e.g. `dir()` or the `TAB` key in Jupyter Notebook) to figure out the names of attributes that you wish to target and modify!
- Another hint for you: the arguments of the function are a bit misleading; we won't actually use the variables ``attr``, ``old``, and ``new`` inside the function. They're included there as necessarily placeholders in order for the python function to map correctly to a [JavaScript callback function](https://docs.bokeh.org/en/latest/docs/reference/models/callbacks.html#bokeh.models.callbacks.CustomJS) (not something we need to concern ourselves with).

In [ ]:
def update_plot(attr, old, new):
    
    """Change properties / attributes of the datasource and title depending on slider value / position."""
    
    yr = slider._____
    new_data = get_geojson(____) #our custom function from before
    geosource.______ = new_data
    p.title._____ = f'Avg. Monthly Temperature Anomaly for Year {yr}'
      

In [ ]:
slider.on_change('value', update_plot)

---

### And finally, some boilerplate code to wrap everything together...
- Wrap the slider in a "widget box", combine it with the figure in a column layout, and add it all to the current document. 

In [ ]:
from bokeh.layouts import widgetbox, column
from bokeh.io import curdoc

layout = column(p,widgetbox(slider))
curdoc().add_root(layout)

In [ ]:
show(layout)

---
---
---

**To view this application in interactive mode you need to set up a local Bokeh server.**

**In the terminal, run:**

``bokeh serve --show <name_of_notebook>.ipynb``

---
---
---

**More Hints**:
- If you're having trouble getting your interactive map working properly, **try exporting your Jupyter Notebook code to a python script.**
    - Clean up your code, remove unnecessary lines, get rid of comments / markdown!
    - Afterwards you can run the bokeh server from the python script:
        - `bokeh serve --show <name_of_script>.py`

---
---
---

### Bonus / Follow-up Questions:
- Add a hover tool (so data is shown when the mouse hovers over a country).
    - `hover = HoverTool(tooltips = [ ('Country','@country'), ('Temp. Anomaly', '@monthly_anomaly')])`
    - `p.tools.append(hover)`
- Any other cool widgets you can think of?
- Get more data up through 2019/2020.
    - Any data source / API where you might be able to get this?
- Create predictions through 2050, and add them to the visualization.
- **Why does the data load slowly, and how could we improve the speed?**